## Keyword Extractor

[KE](https://github.com/protonx-tf-05-projects/vn-extract-keywords/blob/main/generate_keywords.py)

In [ ]:
!pip3 install keyphrase-vectorizers

In [ ]:
!pip3 install py_vncorenlp sentence-transformers

In [ ]:
import py_vncorenlp
import os
from sentence_transformers import SentenceTransformer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

stop_words = []
with open('../data/vietnamese-stopwords.txt', encoding='utf8') as f:
    for line in f:
        stop_words.append(line.strip())

doc = 'Túi xách là một trong những vật dụng cần thiết của hầu hết chị em phụ nữ mỗi khi ra đường.'

def removeStopWords(o_sen):
    words = [word for word in o_sen.split() if word not in stop_words]
    return " ".join(words)

py_vncorenlp.download_model(save_dir=os.path.abspath('./vncorenlp'))

# Load the word and sentence segmentation component
rdrsegmenter = py_vncorenlp.VnCoreNLP(annotators=["wseg"], save_dir=os.path.abspath('./vncorenlp'))

doc_segmented = rdrsegmenter.word_segment(doc)
# Extract candidate words/phrases

count = CountVectorizer(ngram_range=(1,1)).fit([removeStopWords(doc_segmented[0])])
candidates = count.get_feature_names()

model = SentenceTransformer('distiluse-base-multilingual-cased-v2')

doc_embedding = model.encode([doc])
candidate_embeddings = model.encode(candidates)


top_n = 10
distances = cosine_similarity(doc_embedding, candidate_embeddings)
keywords = [candidates[index] for index in distances.argsort()[0][-top_n:]]

print(keywords)

In [ ]:
import underthesea
import os
from sentence_transformers import SentenceTransformer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

stop_words = []
with open('../data/vietnamese-stopwords.txt', encoding='utf8') as f:
  for line in f:
    stop_words.append(line.strip().replace(' ', '_'))

def remove_vie_stop_words(o_sen):
  o_sen = o_sen.lower()
  words = [word for word in o_sen.split() if word not in stop_words]
  # print(words)
  return " ".join(words)


In [ ]:
doc = '''Một nền tảng chung quy chính bản phần mềm dựa trên đám mây cho thị trường giáo dục chuyên nghiệp. 
Công ty khởi nghiệp này đang xây dựng cơ sở dữ liệu biểu đồ để phân tích dữ liệu, dữ liệu không gian địa lý 
và mô hình ngữ nghĩa. Họ cho biết nó sẽ cho phép các giảng viên phân tích sinh viên tốt hơn và cung cấp 
các công cụ giảng dạy tốt hơn.'''

In [ ]:
model = SentenceTransformer('distiluse-base-multilingual-cased-v2')

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
from keyphrase_vectorizers import KeyphraseCountVectorizer # No Vietnamese Spacy Pipeline

doc_segmented_list = [underthesea.word_tokenize(sentence, format='text') for sentence in doc.split('.')]
# doc_segmented = underthesea.word_tokenize(doc, format='text').split(' ')
# Extract candidate words/phrases

count = CountVectorizer(ngram_range=(1,1)).fit([remove_vie_stop_words(doc_segmented) for doc_segmented in doc_segmented_list])
candidates = count.get_feature_names_out()
# print(candidates)

# model = SentenceTransformer('keepitreal/vietnamese-sbert')

doc_embedding = model.encode(doc.split('.'))
candidate_embeddings = model.encode(candidates)

top_n = 10
distances = cosine_similarity(doc_embedding, candidate_embeddings)

#ndarray
keywords = [candidates[index] for index in distances.argsort()[0][-top_n:]]

print(keywords)
# ['cơ_sở_dữ_liệu', 'không_gian', 'giảng_dạy', 'dựa', 'nền_tảng', 'đám', 'phần_mềm', 'mô_hình_ngữ', 'giáo_dục', 'mây']

In [ ]:
from transformers import AutoTokenizer, AutoModel

# https://huggingface.co/keepitreal/vietnamese-sbert
# Load model from HuggingFace Hub
# tokenizer = AutoTokenizer.from_pretrained('keepitreal/vietnamese-sbert')
# model = AutoModel.from_pretrained('keepitreal/vietnamese-sbert')
model = SentenceTransformer('keepitreal/vietnamese-sbert')

In [ ]:
import torch
from transformers import AutoModel, AutoTokenizer

phobert = AutoModel.from_pretrained("vinai/phobert-base")
tokenizer = AutoTokenizer.from_pretrained("vinai/phobert-base")

In [ ]:
from underthesea import word_tokenize
import numpy as np

def phobert_sentence_embedding(raw_sent):
  # Word Segmented raw_sent
  text = word_tokenize(raw_sent, format="text")
  input_ids = torch.tensor([tokenizer.encode(line)])

  #disables gradient calculation.
  with torch.no_grad():
    features = phobert(input_ids).last_hidden_state[:,0,:] # Models outputs are now tuples

  # print(features.shape)
  # torch.Size([1, 768])
  return np.to_array(features)

In [ ]:
phobert_sentence_embedding("Chào em")

In [ ]:
def get_features_from_doc(raw_doc, word2vec) -> list:
  doc_segmented_list = [underthesea.word_tokenize(sentence, format='text') for sentence in raw_doc.split('.')]
  count = CountVectorizer(ngram_range=(1,1))
  matrix = count.fit_transform([remove_vie_stop_words(doc_segmented) for doc_segmented in doc_segmented_list])
  candidates = count.get_feature_names_out()

  words_embed = []
  for candidate in candidates:
    # try:
    #   word_embedding = word2vec[candidate]
    # except:
      word_embedding = phobert_sentence_embedding(candidate)[0]
      words_embed.append(word_embedding)


  return candidates, torch.tensor(words_embed)
  # return candidates

In [ ]:
# https://github.com/sonvx/word2vecVN?fbclid=IwAR3oRcIyVSajJrolyQ2wJXvj5p1AfxGETZgJtkv0QpJTiZY6QvX3Y6tkhrk

from gensim.models import KeyedVectors
from gensim import models

word2vec_path='word2vec/baomoi.model.bin'
word_vectors = models.KeyedVectors.load_word2vec_format(word2vec_path, binary=True)

In [ ]:
word_vectors["văn_bản"]

In [ ]:
torch.tensor(word_vectors["chào"])
# torch.Size([400])

In [ ]:
from underthesea import word_tokenize
from sklearn.metrics.pairwise import cosine_similarity

# https://medium.com/@eskandar.sahel/exploring-feature-extraction-techniques-for-natural-language-processing-46052ee6514
def keyword_extractor(raw_doc, word2vec, top_n = 5):
  candidates, candidates_embedding = get_features_from_doc(raw_doc, word2vec)
  sentences = raw_doc.split('.')
  keywords = set([])
  for idx in range(len(sentences)):
    raw_sentence = sentences[idx]
    sentence_embedding = phobert_sentence_embedding(raw_sentence)

    print(sentence_embedding.shape)
    print('---\n', candidates_embedding.shape)
    cosine_similarity(sentence_embedding, candidates_embedding)
    list_cand = [candidates[index] for index in distances.argsort()[idx][-top_n:]]
    for cand in list_cand:
      keywords.add(cand)

  return keywords

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
from keyphrase_vectorizers import KeyphraseCountVectorizer # No Vietnamese Spacy Pipeline
from sklearn.metrics.pairwise import cosine_similarity
import pandas as pd

doc_segmented_list = [underthesea.word_tokenize(sentence, format='text') for sentence in doc.split('.')]
# doc_segmented = underthesea.word_tokenize(doc, format='text').split(' ')
# Extract candidate words/phrases

count = CountVectorizer(ngram_range=(1,1))
matrix = count.fit_transform([remove_vie_stop_words(doc_segmented) for doc_segmented in doc_segmented_list])
candidates = count.get_feature_names_out()
# print("Vocabulary: ", count.vocabulary_)
print("Feature: ", candidates)

# print(counts)

# model = SentenceTransformer('keepitreal/vietnamese-sbert')

doc_embedding = model.encode(doc.split('.'))
# candidate_embeddings = model.encode(candidates)

# doc_embedding = [vie_sentence_embedding(sent) for sent in doc.split('.')]
candidate_embeddings = model.encode(candidates)
print(doc_embedding)
top_n = 5
distances = cosine_similarity(doc_embedding, candidate_embeddings)
# print(distances)
# [ 0.26308233 -0.07585391 -0.18924099 ... -0.1127905  -0.17521721 0.08920351]
#ndarray
keywords = set([])

for num in range(len(doc_embedding)):
  list_cand = [candidates[index] for index in distances.argsort()[num][-top_n:]]
  for x in list_cand:
    keywords.add(x)

print(keywords)
# ['cơ_sở_dữ_liệu', 'không_gian', 'giảng_dạy', 'dựa', 'nền_tảng', 'đám', 'phần_mềm', 'mô_hình_ngữ', 'giáo_dục', 'mây']

In [ ]:
candidate_embeddings = model.encode(candidates)
print(candidate_embeddings)

In [ ]:
keyword_extractor(doc, word2vec=word_vectors)

In [ ]:
def get_candidate_embedding(sentence):
  phobert_sentence_embedding(sentence)